In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


%matplotlib inline

ImportError: No module named pandas

In [ ]:
tweet_df = pd.read_csv('../data/labeled-twitter-hate-speech.csv')

## Checking Things Out

In [ ]:
tweet_df.columns

In [ ]:
tweet_df.orig__golden.unique()

In [ ]:
tweet_df.orig__golden.value_counts()

In [ ]:
tweet_df.does_this_tweet_contain_hate_speech.value_counts()

In [ ]:
tweet_df.info()

In [ ]:
tweet_df.does_this_tweet_contain_hate_speechconfidence.value_counts()

In [ ]:
golden_df = tweet_df[tweet_df['orig__golden'] == True]

In [ ]:
golden_df.does_this_tweet_contain_hate_speech.value_counts()

In [ ]:
golden_df._trusted_judgments.value_counts()

## Golden Tweets

Determined that `golden` tweets represent tweets reviewed by a large number of users, so seems worthwhile to make use of the column as a boolean value. Filling the null values in that column with `False` as a result.

In [ ]:
tweet_df.orig__golden.fillna(False, inplace=True)

In [ ]:
tweet_df.orig__golden.value_counts()

## NLP on Tweet Text

Separating out the tweet text and labels to focus on the NLP techniques first before returning to other values provided with data.

In [ ]:
tweet_df.columns

In [ ]:
tweet_df.rename?

In [ ]:
tweet_df.rename(columns={'does_this_tweet_contain_hate_speech':'labels'}, inplace=True)

In [ ]:
tweet_df.columns

In [ ]:
text_only_cols = ['labels', 'tweet_text']

In [ ]:
text_only_df = tweet_df.copy()[text_only_cols]

In [ ]:
text_only_df['labels'].value_counts()

In [ ]:
def convert_labels(row):
    if row == 'The tweet is not offensive':
        return 'not offensive'
    elif row == 'The tweet uses offensive language but not hate speech':
        return 'offensive'
    else:
        return 'hate'

In [ ]:
text_only_df['labels'] = text_only_df['labels'].apply(convert_labels)

In [ ]:
text_only_df['labels'].value_counts()

In [ ]:
text_only_df.to_csv('../data/labels_and_text_only.csv')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from nltk.stem import PorterStemmer

In [ ]:
class PorterTokenizer(object):
    def __init__(self):
        self.stemmer = PorterStemmer()
    def __call__(self, doc):
        return [self.stemmer.stem(t) for t in word_tokenize(doc)]

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=PorterTokenizer(), encoding='latin-1', ngram_range=(1,3), stop_words='english')

In [ ]:
X = text_only_df['tweet_text'].values
y = text_only_df['labels'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
X_trans = vectorizer.fit_transform(X_train)

In [ ]:
X_trans.shape